In [49]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import time
from tqdm import tqdm

In [44]:
path = os.path.dirname(os.getcwd())
data_path = os.path.join(path, 'data', 'T10I4D100K.dat')
data_path

'd:\\Study\\KTH\\DM\\assignment\\Datamining_HW2\\data\\T10I4D100K.dat'

In [45]:
def read_dataset(
        file: str
) -> list[set[int]]:
    """
    This function reads from a file .dat assuming that on every row of the file there is a basket of items and returns
    the list of the baskets.

    :param file: the path to the input file
    :return: the list of baskets
    """

    with open(file, "r") as f:
        baskets: list[set[int]] = list(
            map(
                lambda basket: set([int(item_id) for item_id in basket.split()]),
                f.read().splitlines()
            )
        )
    f.close()

    return baskets

In [50]:
baskets = read_dataset(file=data_path)
print(baskets[1])

{704, 834, 581, 39, 205, 814, 401, 120, 825, 124}


In [51]:
C1_raw=dict()
support_thre=len(baskets)*0.01
for basket in baskets:
    for item in basket:
        if item in C1_raw.keys():
            C1_raw[item]+=1
        else:
            C1_raw[item]=1
print(len(C1_raw))

C1=set()
for k,v in C1_raw.items():
    if v>=support_thre:
        C1.add(k)
del C1_raw
print(len(C1))

870
375


In [52]:
C2_raw=dict()
for item1 in C1:
    for item2 in C1:
        if(item1==item2):
            break
        C2_raw[tuple({item1} | {item2})]=0
print(len(C2_raw))
for basket in tqdm(baskets):
    for k in C2_raw.keys():
        if set(k) in basket:
            C2_raw[k]+=1
C2=set()
for k,v in C2_raw.items():
    if v>=support_thre:
        C2.add(set(k))
del C2_raw
print(len(C2))
print(C2)

70125


 34%|███▍      | 34337/100000 [20:30<39:12, 27.91it/s]


KeyboardInterrupt: 

In [ ]:
C3_raw=dict()
for item1 in C1:
    for item2 in C2:
        flag=True
        for t in item2:
            if {t}|item1 not in C2:
                flag=False
                break
        if not flag:
            continue
        C3_raw[item1|item2]=0
print(len(C3_raw))
for basket in baskets:
    for k in C3_raw.keys():
        if k in basket:
            C3_raw[k]+=1
C3=set()
for k,v in C3_raw.items():
    if v>=support_thre:
        C3.add(k)
del C3_raw
print(len(C3))

TypeError: 'int' object is not iterable